In [22]:
import numpy as np
import pandas as pd
import talib as ta
import pandas_datareader as web

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

In [23]:
stockdata_df = pd.read_csv("stockdata.csv")

In [24]:
stockdata_df.head()

,證券代碼,年月日,開盤價,最高價,最低價,收盤價,成交量,收盤價(元),報酬率％
0,2330,2010/1/4,65.0,65.0,64.0,64.9,39511.0,47.28,0.62
1,2317,2010/1/4,152.0,152.5,150.5,151.5,23703.0,63.93,0.00
2,3008,2010/1/4,428.0,428.0,415.5,416.0,1035.0,364.38,-1.19
3,1301,2010/1/4,67.3,67.4,66.6,67.0,5963.0,46.20,-0.45
4,2412,2010/1/4,59.5,59.6,59.1,59.3,14947.0,51.32,-0.34


In [25]:
stockdata_df.columns = ['Code', 'Date', 'Open', 'High', 'Low', 'Close' ,'Volume', 'CloseA', 'ReturnA']

In [26]:
stockdata_df.head()

,Code,Date,Open,High,Low,Close,Volume,CloseA,ReturnA
0,2330,2010/1/4,65.0,65.0,64.0,64.9,39511.0,47.28,0.62
1,2317,2010/1/4,152.0,152.5,150.5,151.5,23703.0,63.93,0.00
2,3008,2010/1/4,428.0,428.0,415.5,416.0,1035.0,364.38,-1.19
3,1301,2010/1/4,67.3,67.4,66.6,67.0,5963.0,46.20,-0.45
4,2412,2010/1/4,59.5,59.6,59.1,59.3,14947.0,51.32,-0.34


In [312]:
stock2330_df = stockdata_df.query('Code == 2408')

In [313]:
stock2330_df.head()

,Code,Date,Open,High,Low,Close,Volume,CloseA,ReturnA
19,2408,2010/1/4,32.50,32.7,31.05,31.70,29620.0,260.12,-3.06
39,2408,2010/1/5,31.90,33.9,31.45,32.65,44158.0,267.91,3.00
59,2408,2010/1/6,32.90,34.0,32.90,33.65,31016.0,276.12,3.06
79,2408,2010/1/7,33.65,34.3,32.75,32.80,28764.0,269.14,-2.53
99,2408,2010/1/8,32.55,32.7,31.20,32.00,27844.0,262.58,-2.44


In [314]:
stock2330_df['RSI'] = ta.RSI(stock2330_df['Close'].values.astype('float64'), 14)

C:\Users\chenminluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [315]:
stock2330_df['ADX'] = ta.ADX(stock2330_df['High'].values.astype('float64'), stock2330_df['Low'].values.astype('float64'), stock2330_df['Close'].values.astype('float64'), timeperiod = 10 )

C:\Users\chenminluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [316]:
stock2330_df['SAR'] = ta.SAR(stock2330_df['High'].values.astype('float64'), stock2330_df['Low'].values.astype('float64'))

C:\Users\chenminluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [317]:
stock2330_df['label'] = (stock2330_df.Close - stock2330_df.Close.shift(1)) > 0

C:\Users\chenminluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [318]:
stock2330_df['Open(-1)'] = stock2330_df['Open'].shift(+1)
stock2330_df['High(-1)'] = stock2330_df['High'].shift(+1)
stock2330_df['Low(-1)'] = stock2330_df['Low'].shift(+1)
stock2330_df['Close(-1)'] = stock2330_df['Close'].shift(+1)
stock2330_df['Volume(-1)'] = stock2330_df['Volume'].shift(+1)
stock2330_df['CloseA(-1)'] = stock2330_df['CloseA'].shift(+1)
stock2330_df['ReturnA(-1)'] = stock2330_df['ReturnA'].shift(+1)
stock2330_df['RSI(-1)'] = stock2330_df['RSI'].shift(+1)
stock2330_df['ADX(-1)'] = stock2330_df['ADX'].shift(+1)
stock2330_df['SAR(-1)'] = stock2330_df['SAR'].shift(+1)

stock2330_df['Open(-2)'] = stock2330_df['Open'].shift(+2)
stock2330_df['High(-2)'] = stock2330_df['High'].shift(+2)
stock2330_df['Low(-2)'] = stock2330_df['Low'].shift(+2)
stock2330_df['Close(-2)'] = stock2330_df['Close'].shift(+2)
stock2330_df['Volume(-2)'] = stock2330_df['Volume'].shift(+2)
stock2330_df['CloseA(-2)'] = stock2330_df['CloseA'].shift(+2)
stock2330_df['ReturnA(-2)'] = stock2330_df['ReturnA'].shift(+2)
stock2330_df['RSI(-2)'] = stock2330_df['RSI'].shift(+2)
stock2330_df['ADX(-2)'] = stock2330_df['ADX'].shift(+2)
stock2330_df['SAR(-2)'] = stock2330_df['SAR'].shift(+2)

stock2330_df['Open(-3)'] = stock2330_df['Open'].shift(+3)
stock2330_df['High(-3)'] = stock2330_df['High'].shift(+3)
stock2330_df['Low(-3)'] = stock2330_df['Low'].shift(+3)
stock2330_df['Close(-3)'] = stock2330_df['Close'].shift(+3)
stock2330_df['Volume(-3)'] = stock2330_df['Volume'].shift(+3)
stock2330_df['CloseA(-3)'] = stock2330_df['CloseA'].shift(+3)
stock2330_df['ReturnA(-3)'] = stock2330_df['ReturnA'].shift(+3)
stock2330_df['RSI(-3)'] = stock2330_df['RSI'].shift(+3)
stock2330_df['ADX(-3)'] = stock2330_df['ADX'].shift(+3)
stock2330_df['SAR(-3)'] = stock2330_df['SAR'].shift(+3)

stock2330_df['Open(-4)'] = stock2330_df['Open'].shift(+4)
stock2330_df['High(-4)'] = stock2330_df['High'].shift(+4)
stock2330_df['Low(-4)'] = stock2330_df['Low'].shift(+4)
stock2330_df['Close(-4)'] = stock2330_df['Close'].shift(+4)
stock2330_df['Volume(-4)'] = stock2330_df['Volume'].shift(+4)
stock2330_df['CloseA(-4)'] = stock2330_df['CloseA'].shift(+4)
stock2330_df['ReturnA(-4)'] = stock2330_df['ReturnA'].shift(+4)
stock2330_df['RSI(-4)'] = stock2330_df['RSI'].shift(+4)
stock2330_df['ADX(-4)'] = stock2330_df['ADX'].shift(+4)
stock2330_df['SAR(-4)'] = stock2330_df['SAR'].shift(+4)

stock2330_df['Open(-5)'] = stock2330_df['Open'].shift(+5)
stock2330_df['High(-5)'] = stock2330_df['High'].shift(+5)
stock2330_df['Low(-5)'] = stock2330_df['Low'].shift(+5)
stock2330_df['Close(-5)'] = stock2330_df['Close'].shift(+5)
stock2330_df['Volume(-5)'] = stock2330_df['Volume'].shift(+5)
stock2330_df['CloseA(-5)'] = stock2330_df['CloseA'].shift(+5)
stock2330_df['ReturnA(-5)'] = stock2330_df['ReturnA'].shift(+5)
stock2330_df['RSI(-5)'] = stock2330_df['RSI'].shift(+5)
stock2330_df['ADX(-5)'] = stock2330_df['ADX'].shift(+5)
stock2330_df['SAR(-5)'] = stock2330_df['SAR'].shift(+5)

C:\Users\chenminluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\chenminluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\chenminluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

C:\Users\chenminluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\chenminluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\chenminluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/index

In [319]:
stock2330_df.dropna(inplace=True)

C:\Users\chenminluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [320]:
X_train, X_test, y_train, y_test = train_test_split(stock2330_df[['Open(-1)', 'High(-1)', 'Low(-1)', 'Close(-1)' ,'Volume(-1)', 'CloseA(-1)', 'ReturnA(-1)', 'RSI(-1)', 'ADX(-1)', 'SAR(-1)', 'Open(-2)', 'High(-2)', 'Low(-2)', 'Close(-2)' ,'Volume(-2)', 'CloseA(-2)', 'ReturnA(-2)', 'RSI(-2)', 'ADX(-2)', 'SAR(-2)', 'Open(-3)', 'High(-3)', 'Low(-3)', 'Close(-3)' ,'Volume(-3)', 'CloseA(-3)', 'ReturnA(-3)', 'RSI(-3)', 'ADX(-3)', 'SAR(-3)', 'Open(-4)', 'High(-4)', 'Low(-4)', 'Close(-4)' ,'Volume(-4)', 'CloseA(-4)', 'ReturnA(-4)', 'RSI(-4)', 'ADX(-4)', 'SAR(-4)', 'Open(-5)', 'High(-5)', 'Low(-5)', 'Close(-5)' ,'Volume(-5)', 'CloseA(-5)', 'ReturnA(-5)', 'RSI(-5)', 'ADX(-5)', 'SAR(-5)']], stock2330_df[['label']], test_size=0.3, random_state=0)

In [321]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [322]:
from sklearn.svm import SVC

In [323]:
clf = SVC()

In [324]:
clf.fit(X_train, y_train)

C:\Users\chenminluo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [325]:
survived_predict = clf.predict(X_test)

In [326]:
from sklearn import model_selection, ensemble, preprocessing, metrics
accuracy = metrics.accuracy_score(y_test, survived_predict)
#print(accuracy)
fpr, tpr, thresholds = metrics.roc_curve(y_test, survived_predict)
auc = metrics.auc(fpr, tpr)
#print(auc)
print('準確率: {}'.format(auc))
print('AUC值: {}'.format(accuracy))

準確率: 0.5
AUC值: 0.5597269624573379
